# DLO-JZ Implémentation du Tensor Parallelism

## Vocabulaire

* TS: Tensor Slicing, autre nom du Tensor Parallelism (pour ne pas avoir de collisions avec TP = Travaux Pratiques :D)
* PP: Pipeline Parallelism
* DDP: Distributed Data Parallelism

## Objet du notebook

Dans ce notebook, vous serez guidé pour implémenter du tensor parallelism (aussi appelé tensor slicing) avec du Pytorch Natif.

Le modèle utilisé que vous devrez paralléliser est un transformer de type BERT simplifié (par exemple il n'a pas de PositionalEncoding, mais c'est pas bien grave). En effet, les modèles de computer vision de type convolution ne sont pas assez larges généralement pour justifier l'usage du TS.
Certains modèles sophistiqués comme CoatNet utilisent à la fois des convolutions et des couches d'attention pour résoudre des problèmes de traitement d'images. Mais cela exigerait de paralléliser plus de types de couches. Pour simplifier les choses je suis parti sur un apprentissage de NLP avec une seule pile d'encodeurs (un BERT donc).

Un schéma du réseau que j'ai implémenté est disponible dans [cet article](https://arxiv.org/pdf/1909.08053.pdf) (Figure 2).

La tâche à résoudre est un dataset de reviews d'IMDb. Vous pouvez observer les csv dans le sous-dossier `data` et constater que l'on a du texte de revues de films, ainsi qu'un label qui dit si la critique est plutôt positive ou négative. On va utiliser un transformer pour automatiser cela. Normalement la tâche est suffisamment simple pour être résolu avec un petit transformer. Mais pour faire apparaître les besoins de tensor slicing, on va utiliser BERT-Base puis BERT-Large.

On ne fera pas d'apprentissage complet. Au début on voudra juste tester nos modifications de code, donc on ne fera que quelques steps.

Dans un second temps, après que le TS sera fonctionnel, je propose des parties _Pour aller plus loin_ qui viseront à remettre de la DDP par dessus, pour avoir un parallélisme 2D. Un parallélisme 3D exigerait également l'ajout de _Pipeline Parallelism_ mais cela demanderait plus de temps, de légèrement retravailler le notebook et les scripts et c'est conceptuellement bien plus difficile que le TS, donc le PP est au-delà des objectifs de ce TP.

## Scripts à disposition

Vous devriez avoir dans ce dossier 7 scripts python :

* dataset.py : Chargement des datasets et instanciation des dataloaders. Vous n'aurez aucun changement à y faire (sauf dans les parties _Pour aller plus loin_).
* model.py : Implémentation du modèle transformer non-distribué. Vous n'aurez aucun changement à y faire.
* pcomm.py (pour process communication car la bibliothèque comm existe déjà :( ): Implémentation des couches de communication nécessaires au TS. Vous travaillerez dessus.
* setup.py : Mes petits outils internes, vous n'avez pas besoin de les consulter, encore moins de les modifier.
* test.py : De courts scripts pour tester chacune des couches que vous implémenterez sans vous embêtez à lancer un entraînement. Chaque test consiste en une forward (comparaison des outputs), une backward (comparaison des gradients) et une step d'optimisation (comparaison des poids après optimisations). Vous n'avez pas besoin de besoin de les consulter, encore moins de les modifier.
* tp_model.py : Une quasi-copie de `model.py`, c'est là que vous travaillerez le plus pour convertir votre transformer avec du TS.
* tp_tensor_parallelism.py : Boucle d'apprentissage et une validation simples. Elle est très similaire au TP sur la DDP. Vous remarquerez dans ce script que j'ai ajouté la précision mixte. Les modifications de base (par exemple `dist.init_process_group` ou `torch.cuda.set_device`) sont déjà incluses car vous avez déjà travaillé sur ces aspects. Vous n'aurez aucun changement  à y faire (sauf dans les parties _Pour aller plus loin_).

## Initialisation et imports

In [ ]:
import math
import os
from datetime import datetime
from idr_pytools import display_slurm_queue, gpu_jobs_submitter, search_log

from setup import read_metrics

In [ ]:
name = "pseudo"
account = "for@a100"
module = "pytorch-gpu/py3/2.4.0"

In [ ]:
def execute(ntasks: int = 2, epochs: int = 1, samples: int = 1024, layers: int = 6, hidden_dim: int = 768, heads: int = 1, batch_size: int = 1, tp: int = 1) -> None:
    """L'argument `tp` fait référence au degré de tensor parallelism. Il n'est pertinent que dans les sous-parties "Pour aller plus loin" """
    execid = datetime.now().strftime("%H:%M:%S")
    command = f"python3 tp_tensor_parallel.py --execid={execid} --epochs={epochs} --samples={samples} --layers={layers} --dim={hidden_dim} --heads={heads} --bsz={batch_size} --tp={tp}"
    jobid = gpu_jobs_submitter(
        command, ntasks, module, name=name, account=account, time_max="00:10:00", qos="qos_gpu-dev", constraint="a100"
    )
    display_slurm_queue(name)
    read_metrics(execid)

def test(classname: str):
    command = f"python3 test.py {classname}"
    jobid = gpu_jobs_submitter(
        command, 2, module, name=name, account=account, time_max="00:10:00", qos="qos_gpu-dev", constraint="a100"
    )
    display_slurm_queue(name)
    %cat {search_log(name, contains=jobid[0])[0]}

## I. Première exécution mono-GPU

_Note: Avec la config ci-dessous (Bert-Base) ça doit rentrer un GPU V100-16GB, mais si vous utilisez un BERT-Large (Layers=24, Heads=16, Hidden_dim=1024), vous devriez avoir un OOM sur un V100-32GB. Dans la suite, on va naviguer entre les deux configurations pour stresser le code sans faire d'OOM_

In [ ]:
execute(ntasks=1, epochs=1, samples=1024, layers=12, hidden_dim=768, heads=8, batch_size=32, tp=1)

## II. Communication en Tensor Parallelism

Note : Pour tout ce qui suit, lorsque vous aurez besoin de faire référence au _rank_ ou _world_size_, vous n'appelerez **pas** directement _idr_torch_. Ces variables seront stockées dans _pcomm.tp_rank_ et _pcomm.tp_degree_. En procédant ainsi, cela simplifiera les parties _Pour aller plus loin_, puisqu'il suffira de changer l'initialisation de ces variables plutôt que de changer chacune de vos modifications. Cette initialisation est faite dans le fichier `pcomm.py` (fonction `init`).

Pour distribuer notre Transformer avec du TS, nous pouvons nous appuyer sur [cet article](https://arxiv.org/pdf/1909.08053.pdf). La figure 3 nous donne toutes les informations dont nous avons besoin pour avancer dans le TP.

Notamment dans ce schéma, les auteurs ont introduit deux opérations de communication qu'ils ont appelées _f_ et _g_. Par soucis de clarté, je les ai respectivement appelées _Duplication_ et _AllReduce_, puisque c'est ce que font effectivement ces couches dans la forward.

**TODO:** Balise 1 -- Dans `pcomm.py`, implémentez les forwards et backwards des fonctions _Duplication_ et _AllReduce_. La fonction _AllGather_ vous est déjà fournie.

**HINT:** Toutes les informations nécessaires sont disponibles dans l'article fourni, le code est donné juste avant la figure.

**SOLUTION:** Dans le fichier `solutions/balise1.py`

## III. Paralléliser chaque couche

On va utiliser nos 3 fonctions _Duplication_, _AllGather_ et _AllReduce_ pour distribuer notre transformer.

### Couche linéaire

Commençons par la couche linéaire. Il y a deux manières de découper une couche linéaire (selon les lignes ou selon les colonnes). Chaque manière implique des communications différentes. Dans ce TP, pour simplifier les choses, toutes les couches linéaires sont sans biais, et on va se focaliser sur un découpage **le long des colonnes**.

**TODO:** Balise 2 -- Dans `tp_model.py`, complétez la classe _ColWiseLinear_. On appelera un découpage le long des colonnes le fait de découper selon la dimension de sortie de la couche. Vous pourrez vous aider des formules dans les slides de cours.

**SOLUTION:** Dans le fichier `solutions/balise2.py`

Vous pouvez tester cette classe via le script suivant (qui fait un forward, compare les outputs, un backward, qui compare les gradients et une step, qui compare les poids après la step).

In [ ]:
test("ColWiseLinear")

### Embedding

**TODO:** Balise 3 -- Faire la même chose avec la couche Embedding

**SOLUTION:** Dans le fichier `solutions/balise3.py`

In [ ]:
test("Embedding")

### MultiHeadSelfAttention

**TODO:** Balise 4 -- Faire pareil avec la couche MultiHeadSelfAttention

**HINT:** Aidez vous de la figure suivante
![megatron_attention](images/megatron_attention.png)

**SOLUTION:** Dans le fichier `solutions/balise4.py`

In [ ]:
test("MultiHeadSelfAttention")

### Assemblons notre transformer

Nos couches _Embedding_ et _MultiHeadSelfAttention_ ont bien été transformées. La couche _ColWiseLinear_ est supposée remplacée une couche linéaire classique.
Notre module _FeedForwardBlock_ continue d'appeler une couche linéaire standard, il faut changer ça.

**TODO:** Balise 5 -- Remplacez les couches linéaires par des couches de type _ColWiseLinear_.

**SOLUTION:** Dans le fichier `solutions/balise5.py`

_Note: Ce module est utilisé à la fois par tous les blocs de Transformer (class Block dans `tp_model.py`) mais je l'ai aussi ré-utilisé dans le classifier final (class Classifier) car j'avais envie de faire un classifier à deux couches (mais j'aurais pu décider autrement)._

_Note: la consommation mémoire devrait avoir fortement baissé. Un BERT-Large doit normalement pouvoir passer sur les V100-16GB._

In [ ]:
execute(ntasks=2, epochs=1, samples=1024, layers=12, hidden_dim=768, heads=8, batch_size=32, tp=2)

## IV. Optimisation des communications

Là on a un petit problème. Chaque couche linéaire fait une communication. Or, quand on a deux couches linéaires consécutives, ce n'est pas forcément obligatoire, on pourrait se contenter d'une seule communication.

**TODO:** Balise 6 -- Implémentez la couche _ColRowLinearPair_.

**HINT:** Aidez-vous de la figure suivante. ![megatron_mlp](images/megatron_mlp.jpeg)

**SOLUTION:** Dans le fichier `solutions/balise6.py`

Testez la:

In [ ]:
test("ColRowLinearPair")

### Assemblons notre nouveau transformer

Notre module _ColRowLinearPair_ doit remplacer le _FeedForwardBlock_ dans les couches _Block_ et _Classifier_.

**TODO:** Balise 7 -- Faire ces modifications.

_Note: Normalement la conso mémoire diminue d'un chouia, et vous allez un peu plus vite, vu que vous faites moins de comm_

In [ ]:
execute(ntasks=2, epochs=1, samples=1024, layers=12, hidden_dim=768, heads=8, batch_size=32, tp=2)

**FIN pour le Tensor Slicing** Ce notebook vous a montré comment on peut faire du Tensor Slicing sur un transformer pour mieux comprendre son fonctionnement. On se rend compte qu'il y a assez peu de modifications à introduire, mais qu'elle nécessite de changer directement les couches du réseau, ce qui peut être compliqué dans certains cas.

Si vous êtes encore motivé et que vous avez encore du temps, vous pouvez faire les parties _Pour aller plus loin_. Elles vous guideront pour remettre de la DDP afin d'avoir un parallélisme 2D.

## V. Pour aller plus loin - Parallélisme 2D

Vu que cette fois, on va mettre en place de la DDP en plus du TS, nos communications collectives n'impliqueront plus tout le monde.

Chaque communication collective permet de ne pas impliquer tous les processus, mais seulement un sous-groupe, réuni dans un communicateur. Avant de mettre le data parallélisme en place,  il faut donc modifier nos communications TS.

**TODO:** Balise 8 -- Créer un sous-communicateur pour le TS. Vous pourrez le stocker dans la variable `tp_grp`. Ce sous-communicateur ne contient pas tous les processus. Le degré de TS est donné par l'argument _tp_ dans la fonction _init_. Le communicateur ne contient tous les processus que si _tp_ = _ntasks_

**HINT:** Elle est cool [cette doc](https://pytorch.org/docs/stable/distributed.html#groups).

**SOLUTION:** Dans le fichier `solutions/balise8.py`

-----------

**TODO:** Balise 9 -- Changez les communications dans `pcomm.py` pour utiliser ce sous-communicateur.

**SOLUTION:** Dans le fichier `solutions/balise9.py`

In [ ]:
execute(ntasks=4, epochs=1, samples=1024, layers=24, hidden_dim=1024, heads=16, batch_size=32, tp=4)

**TODO**: Balise 10 -- Créez un sous-communicateur pour la DDP. Attention, pour les deux sous-communicateurs (_tp_grp_ et _dp_grp_), vous ne pouvez plus leur donner tous les processus, il faut faire un partitionnement.

**SOLUTION:** Dans le fichier `solutions/balise10.py`

In [ ]:
execute(ntasks=4, epochs=1, samples=1024, layers=12, hidden_dim=768, heads=8, batch_size=32, tp=1)

Ça a l'air de tourner mais là on a simplement les communicateurs. On ne fait pas le data parallélisme correctement.

**TODO:** Balise 11 -- Mettre la DDP : fichier `dataset.py` et fichier `tp_tensor_parallélisme.py`.

**SOLUTION:** Dans le fichier `solutions/balise11.py`

In [ ]:
execute(ntasks=4, epochs=1, samples=1024, layers=12, hidden_dim=768, heads=8, batch_size=32, tp=1)

**TODO:** Pour éviter les problèmes avec NCCL, il est crucial que vous mettiez l'argument `use_local_synchronization=True` dans les créations de nouveaux communicateurs quand vous en avez plusieurs, sinon ça va crasher. Modifiez la balise 8 et la balise 10 en conséquence si ce n'est pas déjà fait.

Testez votre parallélisme 2D :)

In [ ]:
execute(ntasks=4, epochs=1, samples=1024, layers=24, hidden_dim=1024, heads=16, batch_size=32, tp=2)

# FINI !!!!!!!!!!

Félicitations vous connaissez maintenant le 2D parallélisme. Comme vous pouvez le constater le TS est un peu fastidieux car il exige d'aller au plus profond du code (pas toujours simple). Mais conceptuellement c'est pas si compliqué et on a pu le mettre en place en seulement une dizaine de balises. Le 3D parallélisme demande l'ajout de PP. C'est possible, il faudrait revoir la manière dont nos `Block` fonctionnent et y faire des communications. Il faudrait aussi faire un nouveau sous-communicateur pour le PP. Le PP est conceptuellement beaucoup plus compliqué à implémenter mais le principe général est le même.

Maintenant si vous le voulez, vous pouvez jouer avec les paramètres pour vérifier que la conso mémoire diminue bien, constater les temps de calcul (vous devriez normalement trouver que le data parallélisme est plus avantageux quand c'est possible), voire même tester que l'accuracy augmente bien en faisant un apprentissage un peu plus long :)

In [ ]:
execute(ntasks=4, epochs=12, samples=1024, layers=24, hidden_dim=1024, heads=16, batch_size=32, tp=2)